In [2]:
import cartoframes
import pandas as pd
pd.options.display.max_columns = 200

import requests as req
import json
import boto3
import io
import datetime
from datetime import datetime
from collections import defaultdict

import sys
import logging
import os
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)


Parameters

In [253]:
datasets_in_wide_form = {} #{wri_id1:rw_id1, wri_id2, rw_id2, ...}

###
## Read from docker data volume - stored in json - can be hooked into API as microservice
###

# Long form table possibilities
datasets = {"cit.022":"6b670396-c52c-430c-b5bb-20693da03b60","cit.025":"d38d0d5c-31b1-47f4-9d2e-d8fba4c7d083","cit.028":"35ce2b98-adbb-4873-b334-d7b1cc542de7","cit.029":"10337db6-8321-445e-a60b-28fc1e114f29","cli.007":"3d2ce960-abda-4c9c-bd29-1929e9ca24c9","cli.008":"3a46f6b4-0eec-49d4-bbfc-e2e8f64e6117","com.006":"2e31a1f3-576b-46b4-84f0-3f0cc399f887","com.007":"fe311144-8c0e-4440-b068-6efd057e0f6a","com.009":"c61c364b-1d68-4dd9-ae3d-76c2a0022280","com.010":"52c55378-0484-48c3-92fc-3ee94d21c716","com.015":"c18a38cd-94ff-48cd-818f-6ffb05992abb","com.019":"5e3a3a9f-7380-47c0-ad84-2c193861e106","com.028":"62c988a7-1e4d-418e-87bf-a743e24209e8","ene.012":"d446a52e-c4c1-4e74-ae30-3204620a0365","ene.021":"2c092793-aa3a-4520-959c-ad48165dcae4","ene.022":"d639909f-bcf3-4875-b8c3-35f030b68ed3","ene.028":"c665f519-eef9-4f67-a8bf-7e3e6dc8bfcd","foo.006":"2034a766-6e8a-416d-b8ab-9b7b3e3abb15","foo.015":"4338471d-881a-475f-8bd9-60c4d48b8e12","foo.019":"8bc79a36-d77e-4ee3-b9bc-c77146cfc503","foo.040":"91ff1359-6680-49bc-8002-20256e999993","foo.041":"ccfb322a-20aa-4132-b58b-0f76acec8f5a","foo.042":"7a551dd8-b59c-4f59-9d50-c92cb61c5799","foo.043":"95b013a3-389a-4367-83b7-c9d68c28c406","for.021":"05b7c688-09ba-4f33-90ea-185a1039df43","soc.004":"bea122ce-1e4b-465d-8b7b-fa11aadd20f7","cit.017":"0303127a-70b0-4164-9251-d8162615d058","soc.005":"a7067e9f-fe40-4338-85da-13a6071c76fe","soc.006":"a89c95c7-0b82-4162-b9d8-cc0205e9f7ec","soc.008":"00abb46f-34e2-4bf7-be30-1fb0b1de022f","soc.015":"e8f53f73-d77c-485a-a2a6-1c47ea4aead9","soc.020":"f8d3e79c-c3d0-4f9a-9b68-9c5ad1f025e4","soc.023":"d3a6b89f-cf5c-40cf-b2b3-ac1c8315c648","soc.029":"7793f46c-a48a-466f-a8ce-ca1a87b7aeed","soc.035":"e7780d53-ad80-45bd-a271-79615ee97a37","soc.036":"8671f536-1979-4b6f-a147-70152fcb44ed","soc.039":"b37048be-9b23-4458-a047-888956c69aa1","soc.040":"37d04efc-0ab2-4499-a891-54dca1013c74","soc.062":"5e69cfac-1f68-4864-a19a-3c1bdb180100","wat.005":"1b97e47e-ca18-4e50-9aae-a2853acca3f0","cli.029":"fa6443ff-eb95-4d0f-84d2-f0c91682efdf"}
datasets_in_wide_form = datasets
datasets_to_geolocate_with_rw_countries = datasets

Code

In [3]:



###
## Set up S3 access ###
### 



#aws_access_key_id = os.environ.get('aws_access_key_id')
#aws_secret_access_key = os.environ.get('aws_secret_access_key')

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/wide_to_long/"

s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)



# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    # Need to set encoding in Python2... default of 'ascii' fails
    df.to_csv(csv_buffer, encoding='utf-8')
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())

    
    
    
    
###
## Set up carto context ###
###

#CARTO_USER = os.environ.get('CARTO_USER')
#CARTO_KEY = os.environ.get('CARTO_KEY')
cc = cartoframes.CartoContext(base_url='https://{}.carto.com/'.format(CARTO_USER),
                              api_key=CARTO_KEY)




###
## Access data from API
###


# Base URL for getting dataset metadata from RW API
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[dset["id"]] = {
        "name":atts["name"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

# Select all Carto datasets on the API:
provider = "cartodb"
carto_ids = (current_datasets_on_api["provider"]==provider)
carto_data = current_datasets_on_api.loc[carto_ids]

logging.info("Number of Carto datasets: " + str(carto_data.shape[0]))

NameError: name 'aws_access_key_id' is not defined

In [304]:
# Load config options
try:
    table_info = json.load(open('./table_info.json', 'r'))
except Exception as e:
    logging.info(e)
    table_info = {}
table_info

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country', 'death_rate_per_100000'],
   'parse_date': True,
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options

In [305]:
### This wrapper allows for only overwriting the 'data' table below... before had to wipe long_data to rerun this

def ddwrapper():
    return defaultdict(lambda : pd.DataFrame())

tables = defaultdict(ddwrapper)

tables['geometry'] = {
    'data':cc.read('wri_countries_a')
}
table_info['geometry'] = {
    'config_options':{}
}

tables['country_aliases'] = {
    'old_data':cc.read('rw_aliasing_countries'),
    'data':cc.read('country_aliases_extended')
}
table_info['country_aliases'] = {
    'config_options':{}
}


###
## This is currently built out to read data already on the API... 
## could be used to prepare data for upload to the API
###



for rw_id, api_id in datasets_in_wide_form.items():
    logging.info(rw_id)

    # Fetch table name
    try:
        table_name = carto_data.loc[api_id]['table_name'].replace('%20', '')
    except:
        msg = '***** ' + rw_id + ' not in carto_data *****'
        table_info[table_name] = {
            'error':msg
        }
        logging.info(msg)

    '''
    if table_name in table_info:
        if 'config_options' in table_info[table_name]:
            if 'prefixes' in table_info[table_name]['config_options']:
                if table_info[table_name]['config_options']['prefixes'] in [[u'--'], [u'already long']]:
                    logging.info('not fetching data for ' + table_name)
                    continue
    '''

    # Fetch data
    try:
        tables[table_name]['data'] = cc.read(table_name)
        if 'config_options' not in table_info[table_name]:
            logging.info('updating table_info for ' + table_name)
            table_info[table_name] = {
                'config_options':{}
            }
    except:
        msg = '***** Problem fetching ' + table_name + ' from Carto server *****'
        if table_name in table_info:
            if 'error' in table_info[table_name]:
                logging.info(table_info[table_name]['error'])
        else:
            table_info[table_name] = {
                'error':msg
            }
            logging.info(msg)


for name, info in tables.items():
    logging.info('table name: ' + name)
    try:
        logging.info('table shape: ' + str(info['data'].shape))
    except:
        logging.info(info['error'])

table_info['soc_023_fragile_states_index'] = {
    'config_options':{'prefixes':['rank', 'score'], 'country_code':False, 'country_name':'country',
                     'id_cols':['country']}
}

table_info['ene_022_se4all_tracking_goals']['config_options']['id_cols'] = ['series_name', 'series_code', 
                                                                            'country_name', 'country_code']

table_info['foo_006_effects_climate_change_food_production']['config_options']['id_cols'] = ['bls_2_countries__sres__abbrevname', 
                                                                                            'fips_code', 'iso3v10',
                                                                                            'wh_2000', 'ri_2000', 'mz_2000',
                                                                                            'country','wh_gr', 'ri_gr', 'mz_gr']
table_info['foo_006_effects_climate_change_food_production']['config_options']['country_code'] = 'iso3v10'
table_info['foo_006_effects_climate_change_food_production']['config_options']['country_name'] = 'country'

table_info['soc_023_fragile_states_index']['config_options']['_incl'] = True
table_info['soc_023_fragile_states_index']['config_options']['country_code'] = False
table_info['soc_023_fragile_states_index']['config_options']['country_name'] = 'country'
table_info['soc_023_fragile_states_index']['config_options']['id_cols'] = ['country']

DEBUG:urllib3.connectionpool:Resetting dropped connection: wri-rw.carto.com
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v2/sql?q=SELECT+%2A+FROM+%22wri_countries_a%22&skipfields=the_geom_webmercator&client=cartoframes_0.3.0b5&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v2/sql?q=SELECT+%2A+FROM+%22rw_aliasing_countries%22&skipfields=the_geom_webmercator&client=cartoframes_0.3.0b5&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v2/sql?q=SELECT+%2A+FROM+%22country_aliases_extended%22&skipfields=the_geom_webmercator&client=cartoframes_0.3.0b5&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
INFO:root:cit.022
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v2/sql?q=SELECT+%2A+FROM+%22cit_022_road_traffic_deaths_by_country%22&skipfields=the_geom_webmerc

In [306]:
table_info

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country', 'death_rate_per_100000'],
   'parse_date': True,
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options

In [57]:
# Add config options
for name, info in tables.items():
    logging.info(name)
    logging.info(list(info['data'].columns))
    print

    ## To determine id columns to use, uncomment
    '''
    if not table_info[name]['config_options'].get('id_cols', False):   
    id_cols = input('id columns? separate by comma with a space')
    table_info[name]['config_options']['id_cols'] = [col.strip() for col in id_cols.split(',')]
    '''
    
    ## To assign prefixes for value column names, uncomment
    '''
    prefixes = input('time column prefixes?')
    table_info[name]['config_options']['prefixes'] = [pfx.strip() for pfx in prefixes.split(',')]
    '''

    ## To determine whether the value column names have a _ after the prefix, uncomment
    '''
    _included = input('Is there a _ after the prefix? type anything for True, leave blank for False')
    if _included != '':
        table_info[name]['config_options']['_incl'] = True
    else:
        table_info[name]['config_options']['_incl'] = False
    '''

    ## To determine country identifying info column, uncomment
    '''
    if table_info[name]['config_options'].get('prefixes', False) == ['already long']:
        country_code = input('country code column? leave blank if needs country code added')
        if country_code != '':
            table_info[name]['config_options']['country_code'] = country_code
        else:
            table_info[name]['config_options']['country_code'] = False

        country_name = input('country name column? leave blank if needs country name added')
        if country_name != '':
            table_info[name]['config_options']['country_name'] = country_name
        else:
            table_info[name]['config_options']['country_name'] = False
    '''
    
    ## Auto parse datetime
    '''
    parse_datetime = input('auto parse datetime? type anything for True, leave blank for False')
    if parse_datetime != '':
        table_info[name]['config_options']['parse_date'] = True
    else:
        table_info[name]['config_options']['parse_date'] = False
    '''

INFO:root:geometry
INFO:root:['_primary', 'abbrev', 'adm0_a3', 'continent', 'formal_nam', 'iso_a2', 'iso_a3', 'iso_n3', 'iso_short', 'name', 'name_sort', 'postal', 'short_name', 'sov_a3', 'sovereign', 'the_geom', 'type', 'un_ar', 'un_en', 'un_es', 'un_fr', 'un_n3', 'un_region', 'un_ru', 'un_subregi', 'un_zh', 'wb_a2', 'wb_a3', 'wb_region']
INFO:root:country_aliases
INFO:root:['alias', 'index', 'iso', 'name', 'the_geom']
INFO:root:cit_022_road_traffic_deaths_by_country
INFO:root:['cartodb_georef_status', 'country', 'death_rate_per_100000', 'estimated_number_of_road_traffic_deaths_2013', 'the_geom']
INFO:root:cit_025_urban_population
INFO:root:['cartodb_georef_status', 'country_code', 'country_name', 'indicator_code', 'indicator_name', 'the_geom', 'yr_1960', 'yr_1961', 'yr_1962', 'yr_1963', 'yr_1964', 'yr_1965', 'yr_1966', 'yr_1967', 'yr_1968', 'yr_1969', 'yr_1970', 'yr_1971', 'yr_1972', 'yr_1973', 'yr_1974', 'yr_1975', 'yr_1976', 'yr_1977', 'yr_1978', 'yr_1979', 'yr_1980', 'yr_1981', 'y

Now we know what we're working with, we can get to work!

In [58]:
## Save this info for working with later
json.dump(table_info, open('./table_info.json', 'w'))
table_info = json.load(open('./table_info.json', 'r'))
table_info

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country'],
   'parse_date': True,
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options': {'_incl': True,
   'co

In [250]:
# Use known prefixes to reformat tables
def pick_value_col(col, pfx, use_):
    if use_:
        if pfx+'_' in col:
            return True
        else:
            return False
    else:
        if pfx in col:
            return True
        else:
            return False

def pick_value_wrapper(col, pfx, use_, xtra):
    return pick_value_col(col,pfx,use_) & (len(col)==xtra+len(pfx))
    
def prepare_date(date, pfx, use_, parse_date):
    if use_:
        if parse_date:
            dt = parser.parse(date[date.index(pfx) + len(pfx) + 1:])
            dt = dt.replace(month=1)
            return dt.replace(day=1)
        else:
            return date[date.index(pfx) + len(pfx) + 1:]
    else:
        if parse_date:
            dt = parser.parse(date[date.index(pfx) + len(pfx):])
            dt = dt.replace(month=1)
            return dt.replace(day=1)
        else:
            return date[date.index(pfx) + len(pfx):]

In [307]:
for name, info in tables.items():
    
    if not table_info[name]['config_options'].get('prefixes', False):
        continue
    if table_info[name]['config_options']['prefixes'] == ['already long']:
        logging.info('transfering ' + name + ', already in long form')
        tables[name]['long_data'] = info['data']
        continue
    if table_info[name]['config_options']['prefixes'] == ['--']:
        logging.info('skipping ' + name + ', requires more manual approach')
        continue

    logging.info('processing ' + name)
    logging.info('options:')
    logging.info(table_info[name]['config_options'])
    
    prefixes = table_info[name]['config_options'].get('prefixes', [])
    id_cols = table_info[name]['config_options'].get('id_cols', [])
    use_ = table_info[name]['config_options'].get('_incl', False)
    parse_date = table_info[name]['config_options'].get('parse_date', False)

    logging.info('initial shape: ' + str(info['data'].shape))
    
    df = pd.DataFrame(columns = id_cols + ['variable'])
    
    for pfx in prefixes:
        logging.debug('working on pfx ' + pfx)
        
        if any([ds in name for ds in ['soc_015']]):
            value_cols = [col for col in info['data'].columns if pick_value_wrapper(col, pfx, use_, 5)]
        elif any([ds in name for ds in ['foo_006']]):
            value_cols = [col for col in info['data'].columns if pick_value_wrapper(col, pfx, use_, 4)]
        else:
            value_cols = [col for col in info['data'].columns if pick_value_col(col, pfx, use_)]
        logging.debug('columns pulled: ' + str(value_cols))

        _df = pd.melt(info['data'], id_vars=id_cols, value_vars=value_cols)
        _df['variable'] = [prepare_date(date, pfx, use_, parse_date) for date in _df['variable']]

        col_names = [pfx+'_data' if col=='value' else col for col in _df.columns]
        _df.columns = col_names

        df = df.merge(_df, on=id_cols  + ['variable'], how='outer')
        logging.debug('intermediate df shape: ' + str(df.shape))

    logging.info('final shape of ' + name + ': ' + str(df.shape))

    new_cols = ['datetime' if col=='variable' else col for col in df.columns]
    df.columns = new_cols
    logging.info('final columns: ' + str(df.columns))
    
    tables[name]['long_data'] = df

INFO:root:processing cit_022_road_traffic_deaths_by_country
INFO:root:options:
INFO:root:{'_incl': True, 'country_code': False, 'country_name': 'country', 'id_cols': ['country', 'death_rate_per_100000'], 'prefixes': ['estimated_number_of_road_traffic_deaths'], 'parse_date': True}
INFO:root:initial shape: (179, 5)
DEBUG:root:working on pfx estimated_number_of_road_traffic_deaths
DEBUG:root:columns pulled: ['estimated_number_of_road_traffic_deaths_2013']
DEBUG:root:intermediate df shape: (179, 4)
INFO:root:final shape of cit_022_road_traffic_deaths_by_country: (179, 4)
INFO:root:final columns: Index(['country', 'death_rate_per_100000', 'datetime',
       'estimated_number_of_road_traffic_deaths_data'],
      dtype='object')
INFO:root:processing cit_025_urban_population
INFO:root:options:
INFO:root:{'_incl': True, 'country_code': 'country_code', 'country_name': 'country_name', 'id_cols': ['country_code', 'country_name', 'indicator_code', 'indicator_name'], 'prefixes': ['yr'], 'parse_date'

SUCCESS!

Inputs

In [254]:
datasets_to_geolocate_with_rw_countries

{'cit.017': '0303127a-70b0-4164-9251-d8162615d058',
 'cit.022': '6b670396-c52c-430c-b5bb-20693da03b60',
 'cit.025': 'd38d0d5c-31b1-47f4-9d2e-d8fba4c7d083',
 'cit.028': '35ce2b98-adbb-4873-b334-d7b1cc542de7',
 'cit.029': '10337db6-8321-445e-a60b-28fc1e114f29',
 'cli.007': '3d2ce960-abda-4c9c-bd29-1929e9ca24c9',
 'cli.008': '3a46f6b4-0eec-49d4-bbfc-e2e8f64e6117',
 'cli.029': 'fa6443ff-eb95-4d0f-84d2-f0c91682efdf',
 'com.006': '2e31a1f3-576b-46b4-84f0-3f0cc399f887',
 'com.007': 'fe311144-8c0e-4440-b068-6efd057e0f6a',
 'com.009': 'c61c364b-1d68-4dd9-ae3d-76c2a0022280',
 'com.010': '52c55378-0484-48c3-92fc-3ee94d21c716',
 'com.015': 'c18a38cd-94ff-48cd-818f-6ffb05992abb',
 'com.019': '5e3a3a9f-7380-47c0-ad84-2c193861e106',
 'com.028': '62c988a7-1e4d-418e-87bf-a743e24209e8',
 'ene.012': 'd446a52e-c4c1-4e74-ae30-3204620a0365',
 'ene.021': '2c092793-aa3a-4520-959c-ad48165dcae4',
 'ene.022': 'd639909f-bcf3-4875-b8c3-35f030b68ed3',
 'ene.028': 'c665f519-eef9-4f67-a8bf-7e3e6dc8bfcd',
 'foo.006': 

Explore geometries and aliasing table

In [66]:
tables['geometry']['data'].head(1)
tables['geometry']['data'][tables['geometry']['data']['adm0_a3']=='MLI']

,_primary,abbrev,adm0_a3,continent,formal_nam,iso_a2,iso_a3,iso_n3,iso_short,name,name_sort,postal,short_name,sov_a3,sovereign,the_geom,type,un_ar,un_en,un_es,un_fr,un_n3,un_region,un_ru,un_subregi,un_zh,wb_a2,wb_a3,wb_region
cartodb_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115,1,Mali,MLI,Africa,Republic of Mali,ML,MLI,466,Mali,Mali,Mali,ML,Mali,MLI,Mali,0106000020E610000001000000010300000001000000D3...,Sovereign country,يلام,Mali,Malí,Mali (le),466,Africa,Мали,Western Africa,马里,ML,MLI,Sub-Saharan Africa


In [134]:
tables['country_aliases']['data'].head(1)
tables['country_aliases']['data'][tables['country_aliases']['data']['alias']=='france']

,alias,iso,name
101,france,FRA,France


Prep work to get country alias table ready

In [308]:

###
## Configuring the alias table
###

df = tables['country_aliases']['data']
logging.info(df.columns)
try:
    df = df.drop('the_geom', axis=1)
except:
    logging.info('could not delete column `the_geom`')


try:
    df = df.drop('index', axis=1)
except:
    logging.info('could not delete column `index`')


logging.info(df.columns)

## Adding all countries from our wri-bounds shapefile to the alias table

new_aliases = tables['geometry']['data'][['iso_a3', 'name']].copy()
new_aliases['alias'] = new_aliases['name']
cols = ['iso' if col=='iso_a3' else col for col in new_aliases.columns]
cols = [col.strip() for col in cols]
new_aliases.columns = cols

logging.info(df.columns)
logging.info(df.shape)
logging.info(new_aliases.columns)
logging.info(new_aliases.shape)

df = df.append(new_aliases)

## Adding in new aliases identified by Peter

peters_new_aliases = pd.read_csv('aliases_for_longform.csv', header=1)
peters_new_aliases.columns = ['alias', 'name', 'iso']

df = df.append(peters_new_aliases)

# Make all aliases lower case, remove spacing
df['alias'] = [alias.strip().lower().replace(' ','') for alias in df['alias']]

## check / remove duplicates
sum(df.duplicated(subset=['alias']))
sum(df.duplicated(subset=['name']))
sum(df.duplicated(subset=['iso']))

df = df.drop_duplicates()
    
tables['country_aliases']['data'] = df

logging.info('Size of current aliasing table: ' + str(tables['country_aliases']['data'].shape))

INFO:root:Index(['alias', 'index', 'iso', 'name', 'the_geom'], dtype='object')
INFO:root:Index(['alias', 'iso', 'name'], dtype='object')
INFO:root:Index(['alias', 'iso', 'name'], dtype='object')
INFO:root:(305, 3)
INFO:root:Index(['iso', 'name', 'alias'], dtype='object')
INFO:root:(193, 3)
INFO:root:Size of current aliasing table: (305, 3)


In [205]:
df.loc[df['name'].str.lower().str.contains('moldova')]

,alias,iso,name
36,republicofmoldova,MDA,Moldova
150,moldova,MDA,Moldova
975,"moldova,republicof",MDA,Moldova
977,moldova(republicof),MDA,Moldova
38,'republicofmoldova(the)',MDA,Moldova
38,republicofmoldova(the),MDA,Moldova


In [312]:
# Tracking all mis-matched names
missed_names = []
missed_isos = []

alias_info = tables['country_aliases']['data']

for name, info in tables.items():

    if name in ['geometry', 'country_aliases']:
        continue
    if name != '':
        #continue
        pass
    
    logging.info('Processing table ' + name)
    logging.info('Table head:')
    
    ### WARNING: non standardized indices in the data cause problems after the merge step
    if 'long_data' in info:
        data = info['long_data'].copy()
        data.index = list(range(data.shape[0]))
    else:
        data = info['data'].copy()
        data.index = list(range(data.shape[0]))
    
    logging.info(data.head(5))

    c_code = table_info[name]['config_options'].get('country_code', False)
    c_name = table_info[name]['config_options'].get('country_name', False)
    
    logging.info('c_code: ***' + str(c_code) + '***')
    logging.info('c_name: ***' + str(c_name) + '***')
    
    
    # Check if isos match our table
    
    if c_code:
        logging.info('already has an iso3 code, in column ' + c_code)
        
        df = data.merge(alias_info,
                           left_on=c_code,
                           right_on='iso', 
                           how='left')
        
        null_isos = pd.isnull(df['iso'])
        
        no_iso_match = df[null_isos.values]
        logging.info('no match for these isos in the data being processed: ')
        logging.info(no_iso_match[c_code].unique())
        missed_isos.extend(no_iso_match[c_code].unique())
        
    
    
    
    
    # If country name is supplied, check how many match up with alias/name in country_aliases
    if c_name:       
        # Ensure that leading or trailing spaces don't break the match
        data[c_name] = ['North Korea' if name=='Korea, Dem. People\x92s Rep.' else name for name in data[c_name]]
    
        _data = data.copy()
        _data['join_col'] = data[c_name].apply(lambda item: item.strip().lower().replace(' ',''))
    
        data_with_alias = _data.merge(alias_info, 
                                         left_on = 'join_col',
                                         right_on = 'alias',
                                         how='left') 

        null_aliases = pd.isnull(data_with_alias['alias'])             
            
        logging.info('data with alias df:')
        logging.info(data_with_alias.shape)
        logging.info(data_with_alias.head(6))
        logging.info('raw data')
        logging.info(data.shape)
        logging.info(data.head(5))
    
        
        if sum(null_aliases):
            no_alias_match = data_with_alias.loc[null_aliases]
            logging.info('missed aliases, matching on column "alias" of country_aliases')
            logging.info(no_alias_match)
            try:
                logging.info(no_alias_match[c_name].unique())
                missed_names.extend(no_alias_match[c_name].unique())
            except:
                c_name = c_name+'_x'
                logging.info(no_alias_match[c_name].unique())
                missed_names.extend(no_alias_match[c_name].unique())
                
        ### data IS ALTERED HERE

        try:
            data['rw_country_code'] = data_with_alias['iso']
        except:
            data['rw_country_code'] = data_with_alias['iso_y']
            
        try:
            data['rw_country_name'] = data_with_alias['name']  
        except:
            data['rw_country_name'] = data_with_alias['name_y'] 

            
        
        
        
    ### SUCCESS
    logging.info('Final head:')
    logging.info(data.head(5))
    tables[name]['geo_data'] = data.copy()


INFO:root:Processing table cit_022_road_traffic_deaths_by_country
INFO:root:Table head:
INFO:root:      country  death_rate_per_100000   datetime  \
0        Mali                   25.6 2013-01-01   
1  Madagascar                   28.4 2013-01-01   
2    Malaysia                   24.0 2013-01-01   
3     Liberia                   33.7 2013-01-01   
4    Maldives                    3.5 2013-01-01   

  estimated_number_of_road_traffic_deaths_data  
0                        3 920 [3 193 - 4 648]  
1                        6 506 [5 361 - 7 650]  
2                        7 129 [6 050 - 8 209]  
3                        1 448 [1 144 - 1 752]  
4                                           12  
INFO:root:c_code: ***False***
INFO:root:c_name: ***country***
INFO:root:data with alias df:
INFO:root:(179, 8)
INFO:root:      country  death_rate_per_100000   datetime  \
0        Mali                   25.6 2013-01-01   
1  Madagascar                   28.4 2013-01-01   
2    Malaysia              

In [224]:
df = tables['cit_028_percent_urban_slums_edit']['data']
df[df['country_code']=='CHI']

,cartodb_georef_status,country_code,country_name,indicator_code,indicator_name,the_geom,yr_1960,yr_1961,yr_1962,yr_1963,yr_1964,yr_1965,yr_1966,yr_1967,yr_1968,yr_1969,yr_1970,yr_1971,yr_1972,yr_1973,yr_1974,yr_1975,yr_1976,yr_1977,yr_1978,yr_1979,yr_1980,yr_1981,yr_1982,yr_1983,yr_1984,yr_1985,yr_1986,yr_1987,yr_1988,yr_1989,yr_1990,yr_1991,yr_1992,yr_1993,yr_1994,yr_1995,yr_1996,yr_1997,yr_1998,yr_1999,yr_2000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016
cartodb_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37,False,CHI,Channel Islands,EN.POP.SLUM.UR.ZS,Population living in slums (% of urban populat...,None,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,NaN,,,,,NaN,,,,,NaN,,,,,NaN,,NaN,,NaN,,,,,NaN,,


In [314]:
missed_names = list(set(missed_names))
print(len(missed_names))
print(missed_names)


missed_isos = list(set(missed_isos))
print(len(missed_isos))
print(missed_isos)

130
['', 'Cook Islands', 'Virgin Islands', 'Saint-Martin (French part)', 'Middle East & North Africa', 'South Asia', 'Guernsey', 'Curacao', 'New Caledonia', 'Fragile and conflict affected situations', 'IDA total', 'Low income', 'Latin America & Caribbean (excluding high income)', 'Tokelau', 'Macao SAR, China', 'Sint Maarten', 'Netherlands Antilles (former)', 'Sub-Saharan Africa', 'Saint Pierre and Miquelon', 'Pitcairn', 'Lower middle income', 'Channel Islands', 'Montserrat', 'United States Minor Outlying Islands', 'Saint Helena, Ascension and Tristan da Cunha', 'ARUBA', 'HONG KONG SAR, CHINA', 'Post-demographic dividend', 'Macao', 'US Virgin Islands', 'Falkland Islands (Malvinas)', 'French Polynesia', 'Anguilla', 'Saint-Barthélemy', 'WEST BANK AND GAZA', 'St. Martin (French part)', 'East Asia & Pacific (IDA & IBRD countries)', 'Jersey', 'Saint Helena', 'Virgin Islands (U.S.)', 'Arab World', 'World', 'Svalbard and Jan Mayen Islands', 'Northern Mariana Islands', 'Gibraltar', 'Guadeloupe'

In [311]:
for name in missed_names:
    for alias in tables['country_aliases']['data']['alias']:
        if alias.lower().replace(' ', '') in name.lower().replace(' ', ''):
            print(name)
df = tables['country_aliases']['data']  
df.loc[df['name'].str.lower().str.contains('cur')]

netherlandsantilles
macaosar,china
unitedstatesvirginislands
frenchguyana
americansamoa
netherlandsantilles(former)
hongkongsar,china
unitedstatesminoroutlyingislands


,alias,iso,name


In [263]:
for iso in missed_isos:
    for iso in tables['geometry']['data']['adm0_a3']:
        if alias.lower().replace(' ', '') in name.lower().replace(' ', ''):
            print(name)
            
for iso in missed_isos:
    for iso in tables['country_aliases']['data']['iso']:
        if alias.lower().replace(' ', '') in name.lower().replace(' ', ''):
            print(name)

In [1]:
write_to_S3(tables['country_aliases']['data'],s3_bucket,s3_folder+'country_aliases')

NameError: name 'write_to_S3' is not defined

In [315]:
cc.write(tables['country_aliases']['data'], 'country_aliases_extended', overwrite=True)

DEBUG:urllib3.connectionpool:Resetting dropped connection: wri-rw.carto.com
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "POST /api/v1/imports?type_guessing=false&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/20568e84-1459-4119-ab3f-b170afd59b3a?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/20568e84-1459-4119-ab3f-b170afd59b3a?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/20568e84-1459-4119-ab3f-b170afd59b3a?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v2/sql?q=%0A++++++++++++++++++++++++++++DROP+TABLE+IF+EXISTS+country_aliases_extended%3B%0A++++++++++++++++++++++++++++ALTER+TABLE+count

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/country_aliases_extended


In [316]:
for name, info in tables.items():
    if name in ['geometry', 'country_aliases']:
        continue
        #pass
    if name != '':
        #continue
        pass
    
    logging.info(name)
    for key, data in info.items():
        if key == 'geo_data':
            logging.info(data.head())
            write_to_S3(data,s3_bucket,s3_folder+name+'_long')
            logging.info('saved ' + name + ' long data to s3')
            cc.write(data, name + '_georefed_&_longform', overwrite=True)
            logging.info('saved ' + name + ' long data to Carto')
            #logging.info('failed to write table ' + name)

INFO:root:cit_022_road_traffic_deaths_by_country
INFO:root:      country  death_rate_per_100000   datetime  \
0        Mali                   25.6 2013-01-01   
1  Madagascar                   28.4 2013-01-01   
2    Malaysia                   24.0 2013-01-01   
3     Liberia                   33.7 2013-01-01   
4    Maldives                    3.5 2013-01-01   

  estimated_number_of_road_traffic_deaths_data rw_country_code rw_country_name  
0                        3 920 [3 193 - 4 648]             MLI            Mali  
1                        6 506 [5 361 - 7 650]             MDG      Madagascar  
2                        7 129 [6 050 - 8 209]             MYS        Malaysia  
3                        1 448 [1 144 - 1 752]             LBR         Liberia  
4                                           12             MDV        Maldives  
INFO:root:saved cit_022_road_traffic_deaths_by_country long data to s3
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartofr

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/cit_022_road_traffic_deaths_by_country_georefed__longform


INFO:root:saved cit_025_urban_population long data to s3
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartoframes/context.py:241: UserWarning: Table will be named `cit_025_urban_population_georefed__longform`
  warn('Table will be named `{}`'.format(table_name))
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "POST /api/v1/imports?type_guessing=false&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/747a884f-a3d7-407a-b94c-a0cd45101d06?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/747a884f-a3d7-407a-b94c-a0cd45101d06?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/747a884f-a3d7-407a-b94c-a0cd45101d06?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 2

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/cit_025_urban_population_georefed__longform


INFO:root:saved cit_028_percent_urban_slums_edit long data to s3
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartoframes/context.py:241: UserWarning: Table will be named `cit_028_percent_urban_slums_edit_georefed__longform`
  warn('Table will be named `{}`'.format(table_name))
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "POST /api/v1/imports?type_guessing=false&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/96a5abe6-d0f1-4047-a4ac-13d6b88f537a?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/96a5abe6-d0f1-4047-a4ac-13d6b88f537a?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/96a5abe6-d0f1-4047-a4ac-13d6b88f537a?api_key=65efbcc00e9591f334fcad66b3d9515228a7

...

INFO:root:saved foo_040_fisheries_vuln_climat long data to s3
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/cartoframes/context.py:241: UserWarning: Table will be named `foo_040_fisheries_vuln_climat_georefed__longform`
  warn('Table will be named `{}`'.format(table_name))


The following columns were changed in the CARTO copy of this dataframe:
rcp26___2016_2050 -> rcp26__2016_2050
rcp26___2066_2100 -> rcp26__2066_2100
rcp45___2016_2050 -> rcp45__2016_2050
rcp45___2066_2100 -> rcp45__2066_2100
rcp85___2016_2050 -> rcp85__2016_2050
rcp85___2066_2100 -> rcp85__2066_2100


DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "POST /api/v1/imports?type_guessing=false&api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/39a5d61e-c7cd-48b9-8ce1-d5ebb65fecae?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/39a5d61e-c7cd-48b9-8ce1-d5ebb65fecae?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/39a5d61e-c7cd-48b9-8ce1-d5ebb65fecae?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/39a5d61e-c7cd-48b9-8ce1-d5ebb65fecae?api_key=65efbcc00e9591f334fcad66b3d9515228a7deef HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://wri-rw.carto.com:443 "GET /api/v1/imports/39a5d61e-c7cd-48b

Table successfully written to CARTO: https://wri-rw.carto.com/dataset/foo_040_fisheries_vuln_climat_georefed__longform


KeyboardInterrupt: 